# GlobalAveragePooling (GAP)
- Feature map의 채널별로 평균값을 추출 1 x 1 x channel 의 Feature map을 생성
- `model.add(keras.layers.GlobalAveragePooling2D())`
![image-2.png](attachment:image-2.png)

- Feature Extraction layer에서 추출한 Feature map을 Classifier layer로 Flatten해서 전달하면 많은 연결노드와 파라미터가 필요하게된다. GAP를 사용하면 노드와 파라미터의 개수를 효과적으로 줄일 수 있다.
- Feature map의 채널수가 많을 경우 GAP를 사용하는 것이 효과적이나 채널수가 적다면 Flatten을 사용하는 것이 좋다.
![image-2.png](attachment:image-2.png)

In [20]:
# 이미지 다운로드
import gdown
url = 'https://drive.google.com/uc?id=1nBE3N2cXQGwD8JaD0JZ2LmFD-n3D5hVU'
fname = 'cats_and_dogs_small.zip'
gdown.download(url, fname, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1nBE3N2cXQGwD8JaD0JZ2LmFD-n3D5hVU
To: /content/cats_and_dogs_small.zip
90.8MB [00:00, 234MB/s]


'cats_and_dogs_small.zip'

In [21]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [22]:
## 압축 풀기
!unzip -q ./cats_and_dogs_small.zip -d data/cats_and_dogs_small

replace data/cats_and_dogs_small/test/cats/cat.1500.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
def get_generators():
    '''
    train, validation, test generator를 생성해서 반환.
    train generator는 image 변환 처리
    '''
    train_dir = './data/cats_and_dogs_small/train'
    validation_dir = './data/cats_and_dogs_small/validation'
    test_dir = './data/cats_and_dogs_small/test'  # 상대경로라서 맨앞에 . 이있다.
    train_datagen = ImageDataGenerator(rescale=1/255,
                                       rotation_range=40,
                                       brightness_range=(0.7,1.3),
                                       zoom_range=0.2,
                                       horizontal_flip=True)
    test_datagen = ImageDataGenerator(rescale=1/255) #validation/test에서 사용
    # generator 들 생성
    train_generator = train_datagen.flow_from_directory(train_dir,
                                                        target_size=(150,150),
                                                        batch_size=N_BATCHS,
                                                        class_mode='binary')
    val_generator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size=(150,150),
                                                        batch_size=N_BATCHS,
                                                        class_mode='binary')
    test_generator = test_datagen.flow_from_directory(test_dir,
                                                        target_size=(150,150),
                                                        batch_size=N_BATCHS,
                                                        class_mode='binary')
    return train_generator, val_generator, test_generator



![image.png](attachment:image.png)

## Transfer learning (전이학습)
- 큰 데이터 셋을 이용해 미리 학습된 pre-trained Model의 Weight를 사용하여 현재 하려는 예측 문제에 활용. 
- ### Convolution base(Feature Extraction 부분)만 활용
    - Convolution base는 이미지에 나타나는 일반적인 특성을 파악하기 위한 부분이므로 재사용할 수 있다.
    - Classifier 부분은 학습하려는 데이터셋의 class들에 맞게 변경 해야 하므로 재사용할 수 없다.
- Pretrained Convlution layer의 활용 
    - Feature extraction
        - 학습시 학습되지 않고 Feature를 추출하는 역할만 한다.
    - Fine tuning
        - 학습시 Pretrained Covolution layer도 같이 학습해서 내 데이터셋에 맞춘다.

## Feature extraction
- 기존의 학습된 network에서 fully connected layer를 제외한 나머지 weight를 고정하고 새로운 목적에 맞는 fully connected layer를 추가하여 추가된 weight만 학습하는 방법
- `tensorflow.keras.applications` module이 지원하는  image classification models
    - (https://www.tensorflow.org/api_docs/python/tf/keras/applications)    
![image.png](attachment:image.png)





> ### ImageNet
>    - 웹상에서 수집한 약 1500만장의 라벨링된 고해상도 이미지로 약 22,000개 카테고리로 구성된다.

> ### ILSVRC(ImageNet Large Scale Visual Recognition Challenge) 대회
>   - 2010년 부터 2017년 까지 진행된 컴퓨터 비전 경진대회.
>   - ImageNet의 이미지중 **1000개 카테고리 약 120만장의 학습용이미지, 5만장의 검증 이미지, 15만장의 테스트 이미지를** 이용해 대회를 진행한다.
>   - **2012년** CNN기반 딥러닝 알고리즘인 **AlexNet**이 2위와 큰 차이로 우승하며 이후 딥러닝 알고리즘이 대세가 되었다. 특히 2015년 우승한 ResNet은 0.036의 에러율을 보이며 우승했는데 이는 사람이 에러율이라 알려진 0.05 보다 높은 정확도였다.
>   - ILSVRC에서 우승하거나 좋은 성적을 올린 모델들이 컴퓨터 비전분야 발전에 큰 역할을 해왔으며 이후 다양한 딥러닝 모델의 백본(backbone)으로 사용되고 있다.

![image.png](attachment:image.png)

##  VGG16 모델
- ImageNet ILSVRC Challenge 2014에서 2등한 모델로 Simonyan and Zisserman(Oxford Univ.)에 의해 제안
    - VGGNet이 준우승을 하긴 했지만, 구조의 간결함과 사용의 편이성으로 인해 1등한 GoogLeNet보다 더 각광받았다
- 단순한 구조로 지금까지 많이 사용.
- 총 16개 layer로 구성됨.
- 네트워크 깊이가 어떤 영향을 주는 지 연구 하기 위해 설계된 네트워크로 동일한 kernel size에 convolution의 개수를 늘리는 방식으로 구성됨.
    - 11 layer, 13 layer, 16 layer, 19 layer 의 네트워크를 테스트함. 
    - 19 layer의 성능이 16 layer보다 크게 나아지지 않음
- Filter의 수가 64, 128, 256, 512 두 배씩 커짐 
- 항상 $3 \times 3$ filter, Stride=1, same padding, $2\times 2$ MaxPooling 사용
    - 이전 AlexNet이 5 X 5 필터를 사용했는데 VGG16은 3 X 3 필터 두개를 쌓아 사용했다.
        - 3 x 3 필터 두개를 쌓는 것이 5 x 5  하나는 사용하는 보다 더 적은 파라미터를 사용하며 성능이 더 좋았다.
    - Feature map의 사이즈를 convolution layer가 아닌 Max Pooling 을 사용해 줄여줌.
- VGG16의 단점은 마지막에 분류를 위해 Fully Connected Layer 3개를 붙여 파라미터 수가 너무 많아 졌다. 약 1억4천만 개의 parameter(가중치)중 1억 2천만개 정도가 Fully Connected Layer의 파라미터 임.
![image-3.png](attachment:image-3.png)

## ResNet (Residual Networks)
- 이전 모델들과 비교해 shortcut connection기법을 이용해 Layer수를 획기적으로 늘린 CNN 모델로 ILSVRC 2015년 대회에서 우승을 차지함.

![image.png](attachment:image.png)

- 레이어를 깊게 쌓으면 성능이 더 좋아 지지 않을까? 실제는 Test 셋 뿐만 아니라 Train Set에서도 성능이 나쁘게 나옴.
- Train set에서도 성능이 나쁘게 나온 것은 최적화 문제로 보고, 레이어를 깊게 쌓으면 최적화 하기가 어렵다고 생각함. 
![image-2.png](attachment:image-2.png)

### Idea
![image-3.png](attachment:image-3.png)

- 입력값을 그대로 출력하는 identity block 을 사용하면 성능이 떨어지지는 않는다.
- 그럼 Convloution block을 identity block으로 만들면 최소한 성능은 떨어지지 않고 깊은 Layer를 쌓을 수 있지 않을까?

### Solution
- Residual block
![image.png](attachment:image.png)

![image.png](attachment:image.png)

- 기존 Layer들의 목표는 입력값인 X를 출력값인 Y로 최적의 매핑할 수 있는 함수 H(X)를 찾는 것이다. 그래서 H(X) – Y 가 최소값이 되는 방향으로 학습을 진행하면서 H(X)를 찾음. 그런데 레이어가 깊어지면서 최적화에 어려움으로 성능이 떨어지는 문제가 발생

- ResNet은 layer를 통과해서 나온 값이 **입력값과 동일하게 만드는 것을 목표로 하는 Identity block을** 구성한다.
- Identity block은 입력값 X를 레이어를 통과시켜서 나온 Y에 입력값 X를 더해서 합치도록 구성한다.

$$\large H(x) = F(x) + x\\x: input,\;H(x): output,\;F(x): layer통과값$$ 
 
 
- 목표는 $H(x)$(레이어통과한 값) 가 input인 x와 동일한 것이므로 F(x)를 0으로 만들기 위해 학습을 한다. 
- $F(x)$는 **잔차(Residual)**가 된다. 그리고 잔차인 $F(x)$가 0이 되도록 학습하는 방식이므로 Residual Learning이라고 한다.
- 입력인 x를 직접 전달하는 것을 **shortcut connection** 또는 **identity mapping** 또는 **skip connection** 이라고 한다.
    - 이 shortcut은 파라미터 없이 단순히 값을 더하는 구조이므로 연산량에 크게 영향이 없다.
- 그리고 Residual을 찾는 레이어를 **Residual Block, Identity Block** 이라고 한다.      

### 성능향상
- $H(x) = F(x) + x$ 을 $x$에 대해 미분하면 최소한 1이므로 Gradient Vanishing 문제를 극복한다.
- 잔차학습이라고 하지만 Residual block 은 Convolution Layer와 Activation Layer로 구성되어 있기 때문에 이 Layer를 통과한 Input으로 부터 Feature map을 추출하는 과정은 진행되며 레이어가 깊으므로 다양한 더욱 풍부한 특성들을 추출하게 되어 성능이 향상된다.
  

### ResNet 구조
- Residual block들을 쌓는 구조
    - 일반 Convolution Layer(backbone)을 먼저 쌓고 Identity(Residual) block들을 계속 쌓는다.
- 모든 Identity block은 두개의 3X3 conv layer로 구성됨.
- 일정 레이어 수별로 filter의 개수를 두배로 증가시키며 stride를 2로 하여 downsampling 함. (Pooling Layer는 Identity block의 시작과 마지막에만 적용)
![image.png](attachment:image.png)

![image.png](attachment:image.png)

## Pretrained Model 사용
- tensorflow.keras.applications 패키지를 통해 제공
- 모델이름이 클래스이름
    - VGG16, ResNet153 등등
- 생성자 매개변수
    - `weights`: 모형의 학습된 weight. 기본값- 'imagenet'
        (imagenet에 개,고양이카테고리가 이미 있음)
    - `include_top`: fully connected layer를 포함할지 여부. True 포함시킴, False: 포함 안 시킴
    - `input_shape`: 사용자가 입력할 이미지의 크기 shape. 3D 텐서로 지정. (높이, 너비, 채널). 기본값: (224,224, 3)

In [ ]:
from tensorflow.keras.applications import VGG16, ResNet50V2

conv_base = VGG16(weights= 'imagenet', # imagenet 데이터셋을 "학습된 가중치(파라미터)"사용
                   #내가 학습한 가중치를 사용하고 싶다면    weights 경로를 넣어 주면 된다. 
                  include_top = False, # claasigication (Fullyconnected layer)는 가져오지않겠다.
                  # true 일때는 모델 전체를 갖고옴(분류classification 레이어까지 갖고옴)
                  input_shape =(150,150,3))
# conv_base= ResNet50V2(weights= 'imagenet',
#                       include_top = False,
#                       input_shape=(150,150,3))

# 사용하고자 하는 class 이름만 바꾸면.. 매개변수 거의 똑같다.

58892288/58889256 [==============================] - 1s 0us/step


In [ ]:
conv_base.summary()  # fullyconnected 가없다.. /conv2_block1_out (Add)          

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

## Feature extraction의 두 가지 방법
1. **빠른 추출방식**
    - 예측하려는 새로운 데이터를 위의 `conv_base`에 입력하여 나온 출력값을 numpy 배열로 저장하고 이를 분류 모델의 입력값으로 사용. Convolution operation을 하지 않아도 되기 때문에 빠르게 학습. 하지만 data augmentation 방법을 사용할 수 없음.

2. **받아온 특성 Layer를 이용해 새로운 모델 구현하는 방식**
    - 위의 `conv_base` 이후에 새로운 layer를 쌓아 확장한 뒤 전체 모델을 다시 학습. 모든 데이터가 convolution layer들을 통과해야 하기 때문에 학습이 느림. 단 conv_base의 가중치는 업데이트 되지 않도록 한다. data augmentation 방법을 사용할 수 있음.

### 빠른 특성 추출 방식

-conv_base에서 피처 맵만 추출하고(conv_base는 학습은 안시킨다) , classification layer(Fullyconnect latyer)를 따로 만들어서 학습시킨다~!


- `conv_base`의 predict 메소드로 입력 이미지의 feature를 추출 

In [75]:
# 하이퍼파라미터
LEARNING_RATE = 0.001  
N_EPOCHS = 30
N_BATCHS = 100

In [76]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG16 , ResNet50V2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

np.random.seed(1)
tf.random.set_seed(1)

In [ ]:
# conv_base 를 통과한 최종 ㄴfeature map size를 확인 할때 이렇게 해봐야함 
conv_base = VGG16(weights= 'imagenet',
                  include_top = False, input_shape = (150,150,3))

conv_base.summary()

In [ ]:
# 2,3교시  함수 만드는 부분 다시 보기 
# featurmap 까지만 추출하는 함수를 만든다. 
def extract_featuremap(image_directory, sample_counts):
    """
    매개변수로 받은 디렉토리의 이미지들을 ""Conv_base(VGG16)"" 모델을 통과시켜서 Featuremap들을 추출해서 반환하는 함수.
    [매개변수] 
        image_directory: 이미지 데이터들이 있는 디렉토리
        sample_counts: 특성을 추출할 이미지 개수 
    [반환값]
        튜플 :(featuremap들 , label)

        sample_counts : 몇개의 이미지를 추출할지 ..
    """
    conv_base = VGG16(weights= 'imagenet',
                      include_top = False, input_shape = (150,150,3))
    
    # 결과를 담을 ndarray // VGG16 이아닌 모델을 쓸때 (4,4,512) 이부분 바꿔줘야함.맨마지막 레이어의 featuremap 사이즈로 바꿔야함
    return_features = np.zeros(shape= (sample_counts, 4,4,512)) # featuremap 저장할 비어있는 배열 (conv_base를 통과한).
     # 어떤 모델을 쓰느냐에 따라서 conv_base의 맨마지막의 layer의 output의 shape(=featuremap size)에 맞춘다.
    return_labels = np.zeros(shape= (sample_counts,)) #label들저장/1차원으로 만듦..

    datagen = ImageDataGenerator(rescale= 1./255) # 이미지를 읽어와야지~
    iterator = datagen.flow_from_directory(image_directory,
                                           target_size =(150,150),
                                           batch_size = N_BATCHS,
                                           class_mode = 'binary')
    # datagen.folow_from_directory 니까 이미지와 라벨을 튜플로 묶어서 batch_size 만큼 이터레이터를 반환하는것이다..


    i = 0 # 반복횟수를 저장할 변수.

    for input_batch, label_batch in iterator: #(image, label)* batch 크기(100)
        # input_batch를 conv_base 넣어서 featuremap 을 추출. 
        #모델.predict() - 모델의 레이어들 통과해서 나온 출력결과를 반환.
        fm = conv_base.predict(input_batch)
        # 슬라이싱 해서 batch 사이즈만큼씩 conv_base를 지난 이미지를 넣어준다.
        return_features[i*N_BATCHS:(i+1)*N_BATCHS] = fm
        return_labels[i*N_BATCHS : (i+1)*N_BATCHS] = label_batch

        i+=1
        if i*N_BATCHS >= sample_counts: # 결과를 저장할 배열의 시작 index가 sample_counts보다 크면 반복문 멈추기 
            break
    return return_features, return_labels

In [ ]:
train_dir = '/content/data/cats_and_dogs_small/train'
validation_dir = '/content/data/cats_and_dogs_small/validation'
test_dir ='/content/data/cats_and_dogs_small/test'

#Featuremap 추출

train_features, train_label = extract_featuremap(train_dir, 2000)
validation_features, validation_label = extract_featuremap(validation_dir,1000)
test_features, test_label = extract_featuremap(test_dir, 1000)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
train_features.shape

(2000, 4, 4, 512)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_label.shape

(2000,)

In [ ]:
def create_model():
    #분류기 모델만 생성
    model = keras.Sequential()
    model.add(layers.Input((4,4,512)))
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(256, activation= 'relu'))
    model.add(layers.Dense(1, activation= 'sigmoid'))

    return model

In [ ]:
model = create_model()

model.compile(optimizer= keras.optimizers.Adam(learning_rate= LEARNING_RATE),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 131,585
Trainable params: 131,585
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(train_features, train_label, #conv_base 를 지나온 넘파이배열로 저장된 feature map들..
                    epochs = N_EPOCHS,
                    validation_data = (validation_features,validation_label), # 데이터 셋을 안만들어서 그냥 튜플로 넣어줌.
                    batch_size = N_BATCHS)

Epoch 1/30
20/20 [==============================] - 1s 15ms/step - loss: 0.6257 - accuracy: 0.6221 - val_loss: 0.4309 - val_accuracy: 0.8290
Epoch 2/30
20/20 [==============================] - 0s 7ms/step - loss: 0.4124 - accuracy: 0.8266 - val_loss: 0.3547 - val_accuracy: 0.8480
Epoch 3/30
20/20 [==============================] - 0s 6ms/step - loss: 0.3406 - accuracy: 0.8547 - val_loss: 0.3374 - val_accuracy: 0.8410
Epoch 4/30
20/20 [==============================] - 0s 6ms/step - loss: 0.3055 - accuracy: 0.8692 - val_loss: 0.2835 - val_accuracy: 0.8840
Epoch 5/30
20/20 [==============================] - 0s 6ms/step - loss: 0.2869 - accuracy: 0.8821 - val_loss: 0.2707 - val_accuracy: 0.8870
Epoch 6/30
20/20 [==============================] - 0s 6ms/step - loss: 0.2470 - accuracy: 0.8983 - val_loss: 0.2684 - val_accuracy: 0.8910
Epoch 7/30
20/20 [==============================] - 0s 6ms/step - loss: 0.2444 - accuracy: 0.8955 - val_loss: 0.2781 - val_accuracy: 0.8830
Epoch 8/30
20/20 [=

In [ ]:
# 한개 이미지 추론 
from tensorflow.keras.preprocessing.image import load_img, img_to_array
def predict_cat_dog(path, model, mode = False):
    class_name = ['고양이', '강아지']
    img = load_img(path, target_size = (150,150,3))
    sample = img_to_array(img)[np.newaxis,...]
    sample = sample/255.
    # 만든 모델이 받는 input이 featuremap 이기 때문에 convloution을  거쳐와야한다..
    if mode: # conv_base를 거치도록
        cb = VGG16(include_top= False, weights = 'imagenet', input_shape=(150,150,3))
        sample = cb.predict(sample)

    pred = model.predict(sample)
    pred_class = np.where(pred<0.5, 0 , 1)
    pred_class_name = class_name[pred_class[0,0]]
    return pred, pred_class, pred_class_name



In [ ]:
predict_cat_dog('/content/dog.jpg', model, mode = True)  # conv 를 지나치게 하려고 한다.

(array([[0.95421654]], dtype=float32), array([[1]]), '강아지')

In [ ]:
predict_cat_dog('/content/cat.jpg', model, mode = True)

(array([[0.02184065]], dtype=float32), array([[0]]), '고양이')

### Pretrained Network를 이용해 새로운 모델 구현하는 방식

- Conv_base의 feature extraction 부분에 fully connected layer를 추가하여 모형 생성 
- Conv_base에서 가져온 부분은 학습을 하지 않고 weight를 고정
    -  **Layer.trainable=False**

In [ ]:
LEARNING_RATE = 0.001
N_EPOCHS = 20
N_BATCHS = 100

IMAGE_SIZE = 150

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
def create_model():
    conv_base = VGG16(weights= 'imagenet',
                      include_top = False,
                      input_shape =(IMAGE_SIZE,IMAGE_SIZE,3))
    
    conv_base.trainable = False # 학습시 weight 최적화(update)를 하지 않도록 설정.
    # 사용할때는 모델 compile 전에 실행해야함.. 
    # 순전파 할때만 쓰이고 역전파 할때는 업데이트가 되지 않는다..(여기서는 VGG16 이 )

    
    model = keras.Sequential()
    model.add(conv_base) # 한번에 convbase 구조가 다 들어온것임.웨이트도 학습된게 붙음
    # // 위에거 summary 해서 다시 공부~!

    model.add(layers.GlobalAveragePooling2D()) # Flatten 대신 사용
    model.add(layers.Dense(256, activation ='relu'))

    # 출력
    model.add(layers.Dense(1, activation ='sigmoid'))

    return model

    


In [ ]:
model = create_model()
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss = 'binary_crossentropy', metrics =['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 14,846,273
Trainable params: 131,585
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
train_iterator, validation_iterator, test_iterator = get_generators()

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
history = model.fit(train_iterator,
                    epochs = N_EPOCHS,
                    steps_per_epoch = len(train_iterator),
                    validation_data = validation_iterator,
                    validation_steps = len(validation_iterator))

Epoch 1/20
20/20 [==============================] - 59s 976ms/step - loss: 0.6604 - accuracy: 0.5981 - val_loss: 0.4737 - val_accuracy: 0.8030
Epoch 2/20
20/20 [==============================] - 20s 984ms/step - loss: 0.4886 - accuracy: 0.7944 - val_loss: 0.3737 - val_accuracy: 0.8510
Epoch 3/20
20/20 [==============================] - 20s 980ms/step - loss: 0.4123 - accuracy: 0.8157 - val_loss: 0.3289 - val_accuracy: 0.8670
Epoch 4/20
20/20 [==============================] - 19s 964ms/step - loss: 0.3802 - accuracy: 0.8334 - val_loss: 0.3057 - val_accuracy: 0.8780
Epoch 5/20
20/20 [==============================] - 19s 970ms/step - loss: 0.3703 - accuracy: 0.8357 - val_loss: 0.2968 - val_accuracy: 0.8880
Epoch 6/20
20/20 [==============================] - 19s 970ms/step - loss: 0.3460 - accuracy: 0.8545 - val_loss: 0.3082 - val_accuracy: 0.8690
Epoch 7/20
20/20 [==============================] - 19s 970ms/step - loss: 0.3395 - accuracy: 0.8475 - val_loss: 0.2805 - val_accuracy: 0.8900

## 미세조정(Fine-tuning)
- Pretrained 모델을 내가 학습시켜야 하는 데이터셋(Custom Dataset)에 재학습시키는 것을 fine tunning 이라고 한다.
- 주어진 문제에 더 적합하도록 모델의 가중치들을 조정.

### Fine tuning 전략
![image-2.png](attachment:image-2.png)

- **세 전략 모두 classifier layer들은 train한다.**
- layer  단위로 trainable을 설정해줘서 조절한다.

1. <span style="font-size:1.2em;font-weight:bold">전체 모델을 전부 학습시킨다.(1번)</span>
    - Pretrained 모델의 weight는 Feature extraction 의 초기 weight 역할을 한다.
    - **Train dataset의 양이 많고** Pretrained 모델이 학습했던 dataset과 Custom dataset의 class간의 유사성이 **낮은 경우** 적용.
    - 학습에 시간이 많이 걸린다.
2. <span style="font-size:1.2em;font-weight:bold">Pretrained 모델 Bottom layer들(Input과 가까운 Layer들)은 고정시키고 Top layer의 일부를 재학습시킨다.(2번)</span>
    - **Train dataset의 양이 많고** Pretrained 모델이 학습했던 dataset과 Custom dataset의 class간의 유사성이 **높은 경우** 적용.
    - **Train dataset의 양이 적고** Pretained 모델이 학습했던 dataset과 custom dataset의 class간의 유사성이 **낮은 경우** 적용
3. <span style="font-size:1.2em;font-weight:bold">Pretrained 모델 전체를 고정시키고 classifier layer들만 학습시킨다.(3번)</span>
    - **Train dataset의 양이 적고** Pretrained 모델이 학습했던 dataset과 Custom dataset의 class간의 유사성이 **높은 경우** 적용.
  
  
> custom dataset: 내가 학습시키고자 하는 dataset 

In [ ]:
cb = VGG16(weights= 'imagenet', include_top= False, input_shape=(150,150,3))

In [ ]:
cb.summary()  

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [26]:
# network(모델)을 구성하는 layer들을 추출
layers = cb.layers # 모델을 구성하는 layer 호출 
type(layers), len(layers) # model을 구성하는 layer들을 추출해서 list에 묶어서 반환.

(list, 19)

In [27]:
layers

In [54]:
layers[2]  # 2번인덱스의 layer / layers[2].trainable = False -> 2번 레이어 train 안함.

In [55]:
# 레이어의 이름 
layers[2].name  # 슬라이싱으로 했을때는 에러남..

'block1_conv2'

In [56]:
l = cb.get_layer('block1_conv2')
l

In [57]:
# 모델.get_layer('layer이름') 지정한 이름의 layer를 반환

ㅣ = cb.get_layer('block1_conv2')  # 이름으로 layer를 호출.
ㅣ

In [58]:
# layer의 가중치(weight)를 조회 // layer객체.weights
l_w = l.weights
type(l_w), len(l_w)#,l_w # [weights, bias],

(list, 2)

In [59]:
np.shape(l_w[0]), np.shape(l_w[1]) # 채널이 64개로 나오니까 bias 64가되고 /
#  3*3 사이즈에 입력값이 64개의 채널을 갖고있으니까 필터 1개가 64개로 구성 / 필터 64개 만듦

(TensorShape([3, 3, 64, 64]), TensorShape([64]))

In [63]:
del layers

### Pretrained 모델 Bottom layer들(Input과 가까운 Layer들)은 고정시키고 Top layer의 일부를 재학습

- Conv_base에서 가장 Top부분에 있는 레이어에 대해 fine-tuning.
    - 앞의 layer들은 비교적 일반적이고 재사용 가능한 feature를 학습
    - 너무 많은 parameter를 학습시키면 overfitting의 위험이 있음 (특히 새로운 데이터의 수가 적을 때)

In [ ]:
#block5_conv2,block5_conv3를 trainabel =True 할거임..// 나머지는 freezing (trainabele =False)
# 맨마지막 block3_maxpooling2D는 어차피 학습하는게 없으니까(파라미터가 없으니까) 상관없음.. 

In [79]:
def create_model():
    # VGG16 사용 , block5_conv2,block5_conv3 두개의 convolution layer들을 fine tuning 할것이다.
    conv_base = VGG16(weights= 'imagenet',include_top= False, input_shape=(150,150,3))
    # trainable 설정
    is_trainable = False # 중간에 True로 바꾸기위해 변수처리
    for layer in conv_base.layers: # 레이어들의 리스트를 하나씩 갖고옴
        if layer.name =='block5_conv2': # layer 이름을 이용했음.. True로 바뀐후 그밑에 있는 애들은 True가 적용되면서 trainable이 된다.
            is_trainable = True
        layer.trainable = is_trainable   # 일단 전부 False 로 바꾸면서 원하는 지점에서 True로 바꾼다

    model = keras.Sequential()
    model.add(conv_base)
    model.add(keras.layers.GlobalAveragePooling2D())
    model.add(keras.layers.Dense(256, activation= 'relu'))
    model.add(keras.layers.Dense(1, activation = 'sigmoid'))


    return model


In [80]:
model = create_model()
model.compile(optimizer = keras.optimizers.Adam(learning_rate= LEARNING_RATE), loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_5 ( (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 257       
Total params: 14,846,273
Trainable params: 4,851,201
Non-trainable params: 9,995,072
_________________________________________________________________


In [81]:
train_iterator,validation_iterator,test_iterator = get_generators() # 이미지 갖고옴

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [84]:
N_EPOCHS = 30 
mc_callback = keras.callbacks.ModelCheckpoint('./models/cat_dog_model', monitor= 'val_loss', save_best_only= True)
model.fit(train_iterator,epochs= N_EPOCHS,steps_per_epoch= len(train_iterator),
          validation_data = validation_iterator, validation_steps = len(validation_iterator),
          callbacks = [mc_callback])
# fine 튜닝을 통해 학습한 데이터 때문에 정확도나 loss가 좋아졌다.

Epoch 1/30
 6/20 [========>.....................] - ETA: 5:41 - loss: 1.5309 - accuracy: 0.5201

KeyboardInterrupt: ignored

In [ ]:
best_model = keras.models.load_model('./models/cat_dog_model') # 모델 정장햇고, 경로 지정해주면됨. 4/28 zoom_4강의

In [ ]:
#evlauation
best_model.evaluate(train_iterator)

In [ ]:
best_model.evaluate(test_iterator)

In [ ]:
predict_cat_dog('/content/cat.jpg', best_model, mode=False)

In [ ]:
predict_cat_dog('/content/dog.jpg', best_model, mode=False)

In [ ]:
# 베스트 모델을 계속 쓰고 싶다면 zip 명령어를 사용해서 압축해서 다운받아서 사용하면됨~!